## Project Euler 
#### Questions 31-40

Continuing the quest to solve every Euler question!

---
**Problem 31:** Coin sums

In the United Kingdom the currency is made up of pound (£) and pence (p). There are eight coins in general circulation: 1p, 2p, 5p, 10p, 20p, 50p, £1 (100p), and £2 (200p).

How many different ways can £2 be made using any number of coins?

In [76]:
#Make a list of all coins:
coins = [200,100,50,20,10,5,2,1]

In [78]:
#This is a classic recursion problem!
def ways_possible_change(input_coin_list, amount_to_change):
    '''Input a set of coins with set values and any amount of money, 
    output the number of possible combinations of coins to make that amount.'''
    
    input_coin_list.sort()
    coin_list = input_coin_list[::-1]
    coin_list = [coin for coin in coin_list if coin <= amount_to_change]
    
    #Fully divided ammount
    if amount_to_change == 0:
        return 1
    
    #One coin left
    elif len(coin_list) == 1:
        if amount_to_change % coin_list[0] == 0:
            return 1
        else:
            return 0
    
    #Coin equals amount to change
    elif coin_list[0] == amount_to_change:
        return ways_possible_change(coin_list[1:], amount_to_change)+1
    
    else:
        partitions = amount_to_change // coin_list[0]
        change_left_over = [amount_to_change - coin_list[0]*(i) for i in range(int(partitions)+1)]
        return sum([ways_possible_change(coin_list[1:], amount) for amount in change_left_over])

In [77]:
ways_possible_change(coins, 200)

73682

In [79]:
#For fun, ways to change a dollar:
ways_possible_change([25,10,5,1], 100)

242

---
**Problem 32:** Pandigital Products

We shall say that an n-digit number is pandigital if it makes use of all the digits 1 to n exactly once; for example, the 5-digit number, 15234, is 1 through 5 pandigital.

The product 7254 is unusual, as the identity, 39 × 186 = 7254, containing multiplicand, multiplier, and product is 1 through 9 pandigital. Find the sum of all products whose multiplicand/multiplier/product identity can be written as a 1 through 9 pandigital.

**Some Notation and Notes:** We will use the variables $a,b,c\in\mathbb{Z}^{+}$ for the equation $a\times b = c$ with the text expansion $\{a_i\},\{b_i\},\{c_i\}$ with cardinalities $n_a,n_b,$ and $n_b$, respectively, such that $\{a_i\}\cup\{b_i\}\cup\{c_i\} = \{1,2,\ldots,8,9\}.$ First, I will create some algorithms to check if the trio is a successful match. We obviously can't check everything, so the next step will be to intelligently reduce the search space!

In [64]:
#First define the brute force checkers
successful_set = {'1', '2', '3', '4', '5', '6', '7', '8', '9'}
def num_len(number):
    return len(str(number))
def num_to_list(number):
    return [str(number)[i] for i in range(len(str(number)))]
def possible(number):
    if '0' in num_to_list(number):
        return False
    return len(num_to_list(number)) == len(set(num_to_list(number)))
def check_group(pair):
    total_digits = num_to_list(pair[0])+num_to_list(pair[1])
    return len(total_digits) == len(set(total_digits))
def check_abc(a,b,c):
    if num_len(a) + num_len(b) + num_len(c) != 9:
        return False
    if set(num_to_list(a) + num_to_list(b) + num_to_list(c)) != successful_set:
        return False
    return True
check_abc(39,186,7254), possible(1023), check_group([38,173])

(True, False, False)

First, note that $k_a+k_b > 4$ is a necessary condition since otherwise the product is bounded by $9\times 999 = 8991$ and $99*99 = 9801$ above, neither of which have the trio $(a,b,c)$ with a sum of 9 digits. Second, $k_a + k_b < 6$ since the lower bound $1\times 10000 = 10000$ gives a trio of ten digits. So we have $k_a+k_b = 5$.To avoid redundancy, we will say that $a<b$. That gives us $2\leq a<100$ and $100\leq b < 10000$  since $k_b \geq k_a$ shrinking the search space immensely. So now, we will create a list using this range, filter first all $a,b$ that have duplicate digits independently, then filter $a,b$ with duplicate digits, and then create a product and check that it has unique digits. By filtering, we will have less than 6000 trios to check.

In [72]:
a = [i for i in range(2, 100) if possible(i)]
#Filter double numbers
four_digits = [num for num in range(1000, 10000) if possible(num)]
three_digits = [num for num in range(100, 1000) if possible(num)]
First_cut = []
for num in a:
    if num_len(num) == 1:
        First_cut += [[num, num_4] for num_4 in four_digits]
    else:
        First_cut += [[num, num_3] for num_3 in three_digits]
print(len(second_cut))
Second_cut = [tuple_ for tuple_ in First_cut if check_pair(tuple_)]
print(len(third_cut))
final_cut = [[pair[0],pair[1],pair[0]*pair[1]] for pair in Second_cut if possible(pair[0]*pair[1])]
print(len(final_cut))
#And finally, brute force the rest
desired_triplets = [trio for trio in final_cut if check_abc(trio[0],trio[1],trio[2])]

60480
28560
5566


In [73]:
#We have some more duplicates:
final_list_of_c = list(set([trio[2] for trio in desired_triplets]))
print("Number of Products: {}".format(len(final_list_of_c)))
print("Sum of Products: {}".format(sum(final_list_of_c)))
final_list_of_c

Number of Products: 7
Sum of Products: 45228


[5346, 5796, 6952, 7852, 4396, 7632, 7254]

---
**Problem 33:** Curious Fractions

The fraction 49/98 is a curious fraction, as an inexperienced mathematician in attempting to simplify it may incorrectly believe that 49/98 = 4/8, which is correct, is obtained by cancelling the 9s.

We shall consider fractions like, 30/50 = 3/5, to be trivial examples.

There are exactly four non-trivial examples of this type of fraction, less than one in value, and containing two digits in the numerator and denominator. If the product of these four fractions is given in its lowest common terms, find the value of the denominator.

**Strategy:** Like every other question, we should first filter out the cases.

In [198]:
#Make all fractions with two digits in numerator/demoninator less than 1
numerators = [i for i in range(10, 99)]
all_fracs = []
for num in numerators:
    all_fracs += ['{}/{}'.format(num, j) for j in range(num+1, 100)]
print("       Number of total fractions: {}".format(len(all_fracs)))

#First cut all the fractions which num/den don't share a common digit
def duplicate_digits(str_frac):
    numerator, denominator = str_frac.split('/')
    common_digits = [digit for digit in denominator if numerator.find(digit) != -1]
    return len(common_digits) != 0
first_cut = [frac for frac in all_fracs if duplicate_digits(frac)]
print("          Number after first cut: {}".format(len(first_cut)))

#Cut any cases with 0's in num or den.
#First, this will cut out all trivial cases
#If the fraction isn't trivial, canceling the remaining values in the non-intellegent way
        #will leave the fraction either equal to 0, or undefined.
def no_zeros(str_frac):
    numerator, denominator = str_frac.split('/')
    if numerator.find('0') + denominator.find('0') > -2:
        return False
    return True
second_cut = [frac for frac in first_cut if no_zeros(frac)]
print("    Number of cases without zero: {}".format(len(second_cut)))

       Number of total fractions: 4005
          Number after first cut: 1377
    Number of cases without zero: 1188


In [212]:
def condition(a,b,ca,cb):
    return (a*cb)/(b*ca) == 1

def test_pairs(fraction):
    #Split the fraction
    a, b = fraction.split('/')
    if b.find(a[0]) != -1:
        if condition(int(a), int(b), int(a[1]), int(b[(b.find(a[0])+1)%2])):
            return True
    if b.find(a[1]) != -1:
        if condition(int(a), int(b), int(a[0]), int(b[(b.find(a[1])+1)%2])):
            return True
    return False 
final_list = [frac for frac in second_cut if test_pairs(frac)]
final_list

['16/64', '19/95', '26/65', '49/98']

In [202]:
num = 1
for x in final_list:
    num *= int(x[:2])
den = 1
for x in final_list:
    den *= int(x[3:])
num, den

(387296, 38729600)

In [213]:
#We will need this function we've used before:
def is_prime(num):
    for i in range(2, num//2+1):
        if abs(num) % i == 0:
            return False
    return True  
primes_under_100 = [number for number in range(2, 10000) if is_prime(number)]
def prime_factorization(number):
    '''Take a number, return a prime factorization.'''
    
    factors = [possible_factor for possible_factor in primes_under_100 if number % possible_factor == 0]
    powers = []
    for factor in factors:
        remainder = number
        power = 1
        while remainder % factor == 0:
            remainder = remainder/factor
            if remainder % factor == 0:
                power += 1
        powers.append(power)  
    return {factors[k]: powers[k] for k in range(len(factors))}
top = prime_factorization(num)
bottom = prime_factorization(den)
top, bottom

({2: 5, 7: 2, 13: 1, 19: 1}, {2: 7, 5: 2, 7: 2, 13: 1, 19: 1})

In [216]:
factored_top = top.copy()
factored_bottom = bottom.copy()
for key in top.keys():
    if key in bottom.keys():
        factored_top[key] = top[key] - min(top[key], bottom[key])
        factored_bottom[key] = bottom[key] - min(top[key], bottom[key])
factored_top, factored_bottom

({2: 0, 7: 0, 13: 0, 19: 0}, {2: 2, 5: 2, 7: 0, 13: 0, 19: 0})

So, the final fraction is 1/100!